# Stage 1 - Make and train the model

In [22]:
import pandas as pd
import numpy as np
import torch

## dataset generation
# lets say we want to train a function that returns max(x,y)

input_data = pd.DataFrame({
    'x': np.random.uniform(-1000, 1000, 10000),
    'y': np.random.uniform(-1000, 1000, 10000)
})
output_data = pd.DataFrame({
    'max': np.maximum(input_data['x'], input_data['y'])
})


df = pd.concat([input_data, output_data], axis=1)

df.describe()

,x,y,max
count,10000.000000,10000.000000,10000.000000
mean,-21.800063,-0.506054,324.312501
std,578.818310,578.531328,474.106109
min,-999.961736,-999.873428,-987.980280
25%,-527.600375,-504.048709,-10.528082
50%,-34.174098,0.665823,399.594383
75%,470.701620,499.911204,729.902342
max,999.789837,999.878718,999.878718


In [23]:
# model definition and training

class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = torch.nn.Linear(2, 10)
        self.linear2 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
    # this method is necessary for the model to be evaluated by poly-lithic

    def evaluate(self, x):
        # x will be a dicrt of keys and values
        # {"x": x, "y": y}
        input_tensor = torch.tensor([x['x'], x['y']], dtype=torch.float32)
        # you may want to do somethinf more complex here
        output_tensor = self.forward(input_tensor)
        # return a dictionary of keys and values
        return {"max": output_tensor.item()}        
    
# model training
model = SimpleModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()
batch_size = 32

# convert data to torch tensors
X = torch.tensor(df[['x', 'y']].values, dtype=torch.float32)
y = torch.tensor(df[['max']].values, dtype=torch.float32)
# training loop
for epoch in range(100):
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        # forward pass
        y_pred = model(X_batch)

        # compute loss
        loss = loss_fn(y_pred, y_batch)

        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')
    

Epoch 0, Loss: 192.95655822753906
Epoch 1, Loss: 25.026445388793945
Epoch 2, Loss: 10.21835708618164
Epoch 3, Loss: 7.42478609085083
Epoch 4, Loss: 5.154231071472168
Epoch 5, Loss: 2.9389894008636475
Epoch 6, Loss: 1.4529979228973389
Epoch 7, Loss: 0.7924959659576416
Epoch 8, Loss: 0.4496413767337799
Epoch 9, Loss: 0.2790074646472931
Epoch 10, Loss: 0.15671026706695557
Epoch 11, Loss: 0.08996041864156723
Epoch 12, Loss: 0.04478834569454193
Epoch 13, Loss: 0.014473375864326954
Epoch 14, Loss: 0.007029773201793432
Epoch 15, Loss: 0.004069437272846699
Epoch 16, Loss: 0.012737715616822243
Epoch 17, Loss: 0.003976777195930481
Epoch 18, Loss: 0.032771702855825424
Epoch 19, Loss: 0.16918373107910156
Epoch 20, Loss: 0.4616890847682953
Epoch 21, Loss: 2.6224358081817627
Epoch 22, Loss: 8.3590087890625
Epoch 23, Loss: 0.04710977524518967
Epoch 24, Loss: 18.396827697753906
Epoch 25, Loss: 2.970618486404419
Epoch 26, Loss: 1.8113993406295776
Epoch 27, Loss: 0.017150355502963066
Epoch 28, Loss: 0.0

In [24]:
# test the model
test_data = pd.DataFrame({
    'x': np.random.uniform(-1000, 1000, 100),
    'y': np.random.uniform(-1000, 1000, 100)
})
test_X = torch.tensor(test_data[['x', 'y']].values, dtype=torch.float32)
test_y = torch.tensor(np.maximum(test_data['x'], test_data['y']), dtype=torch.float32)
test_y_pred = model(test_X)
test_loss = loss_fn(test_y_pred, test_y)
print(f'Test Loss: {test_loss.item()/len(test_X)}')

# print sample
for i in range(10):
    print(f"Input: {test_data.iloc[i].values}, Predicted: {test_y_pred[i].item()}, Actual: {test_y[i].item()}")

Test Loss: 3911.0959375
Input: [-877.3532425   833.82588463], Predicted: 834.4656982421875, Actual: 833.8258666992188
Input: [-440.90725107 -985.49221843], Predicted: -443.4677429199219, Actual: -440.9072570800781
Input: [-133.4418616  -480.41190893], Predicted: -134.64442443847656, Actual: -133.44186401367188
Input: [-466.10806235  774.32364276], Predicted: 775.1988525390625, Actual: 774.3236694335938
Input: [-690.96579231  794.55989219], Predicted: 795.272216796875, Actual: 794.5598754882812
Input: [-231.62883244  933.3487746 ], Predicted: 934.8477783203125, Actual: 933.3487548828125
Input: [ 770.97776864 -222.97240719], Predicted: 771.0821533203125, Actual: 770.977783203125
Input: [-636.76258515 -351.32397158], Predicted: -352.3260192871094, Actual: -351.323974609375
Input: [-670.6501325  -553.75296158], Predicted: -555.0887451171875, Actual: -553.7529907226562
Input: [-441.7944957    63.48388469], Predicted: 63.25518798828125, Actual: 63.483882904052734


/home/gbm96348/nfs_home/ml_env/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# Stage 2 - Write a poly-lithic compatible model wrapper

In [ ]:
# now we need to define the model wrapper and save it.

: 